In [3]:
# max row, columns 셋팅
import pandas as pd
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
import numpy as np
from pandas import DataFrame, Series
import os
import datetime
import re
######################mapping 표 불러오기################

sheet=['Country_map','Channel_map','Product_map','US_channel']

map=Series()
for i in range(0,len(sheet)):
    map[sheet[i]]= pd.read_excel('C:/Users/cnxk/Desktop/dashboard_prac/Map.xlsx',

                              sheet_name = sheet[i], 

                              header = 1, 

                              thousands = ','  )   
    map[sheet[i]]=map[sheet[i]].dropna(axis=1,how='all')#첫째열은 전체 빈행->삭제해주기
    
map['Product_map']['AA Product Model']=map['Product_map']['AA Product Model'].str.upper() #대문자로 통일


######################어도비 데이터 그대로 불러오기####################


sheet=['KPI','Referring Domain','Marketing Channel','URL','Product','Time Spent']

scom=Series()
for i in range(0,len(sheet)):
    scom[sheet[i]]= pd.read_excel('C:/Users/cnxk/Desktop/dashboard_prac/Scom_all_june.xlsx',

                              sheet_name = sheet[i], 

                              header = 1, 
                                  

                              thousands = ','  )   
    
##dimension열 같은 필요없는 컬럼삭제
    col_not_dimension = []
    for j in scom[sheet[i]].columns:
        if bool(re.match('^Dimension',j)) == False:
            col_not_dimension.append(j)
    scom[sheet[i]]=scom[sheet[i]][col_not_dimension]
    
##자이제 Item컬럼의 빈행, 토탈행, low-traffic 행을 지워야한다.    
    scom[sheet[i]]=scom[sheet[i]].dropna(axis=1,how='all')#첫째열은 전체 빈열->삭제해주기
    scom[sheet[i]]=scom[sheet[i]].dropna(how='any')#NA값들어간 행 지우기
    
    
    
#필요없는 행들 지우기, 빈줄은 위에서 다지웠고
#KPI -> item=total
#'Referring Domain'->item.1=total
#'Marketing Channel'->item.1=total
#'URL'->item.1=total,(Low-traffic)
#'Product'->item.1=total
#'Time Spent'->item.2=Total

scom['KPI']=scom['KPI'].loc[ (scom['KPI']['Item']!='Total'),]
scom['Referring Domain']=scom['Referring Domain'].loc[ (scom['Referring Domain']['Item.1']!='Total'),]
scom['Marketing Channel']=scom['Marketing Channel'].loc[ (scom['Marketing Channel']['Item.1']!='Total'),]
scom['URL']=scom['URL'].loc[ (scom['URL']['Item.1']!='Total')
                                                      &(scom['URL']['Item.1']!=r'(Low-traffic)'),]
scom['Product']=scom['Product'].loc[ (scom['Product']['Item.1']!='Total'),]
scom['Time Spent']=scom['Time Spent'].loc[ (scom['Time Spent']['Item.2']!='Total'),]


C:\Users\cnxk\anaconda3\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
C:\Users\cnxk\anaconda3\lib\site-packages\ipykernel_launcher.py:33: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


3       MZ-76E250
4       MZ-V7S2T0
5       MZ-V7S500
6       MZ-76P4T0
7       MZ-76E500
          ...    
3523    MU-PC1T0T
3524    MB-MJ128G
3525     MUF-32AB
3526    MU-PA500R
3527    MU-PC1T0H
Name: Item.1, Length: 3445, dtype: object

In [4]:


##############사이트코드담긴 열, 인덱스로 빼주기
scom['KPI']=scom['KPI'].set_index('Item',drop=True)
scom['Time Spent']=scom['Time Spent'].reset_index(drop=True).set_index('Item.1',drop=True)

a=scom['Time Spent']['[MST] Memory_41 Sites + [MST] Memory - Visit']
b=scom['Time Spent']['[MST] Memory_41 Sites + [MST] Memory - Bounce']
c=scom['Time Spent']['Time Spent per Visit (seconds)']
sub=[x-y for x,y in zip (a,b)]
subT=[x*y for x,y in zip (sub,c)]


scom['Time Spent']['(V-B)']=sub
scom['Time Spent']['(V-B)*T']=subT

#이제 나라별 평균체류시간 구해야한다. 피벗

avg_duration= scom['Time Spent'].pivot_table(values=['(V-B)','(V-B)*T'],
                                                    index=scom['Time Spent'].index,
                                                    aggfunc='sum')
avg_duration
avg_duration=avg_duration[avg_duration['(V-B)']!=0]

avg_duration['Spent Time']=[y/x for x,y in zip (avg_duration['(V-B)'],avg_duration['(V-B)*T'])]

#time spent랑 kpi가 정리 끝,
#두개 붙이면 RAW_KPI 완성~! scom['KPI']에 위의 avg_duration 붙여서 정렬시키고 key값 생성하면 완성

scom['KPI']['Time Spent']=avg_duration['Spent Time']
scom['KPI'].columns=['Visit','Visit - PC','Visit - Mobile','Visitor','Page Views',
                 'Entry','Bounce','SSD','PSSD','Memory Card','USB','PFS','PCD','PF','PD','Time Spent']
scom['KPI']=scom['KPI'][['Visit','Visit - PC','Visit - Mobile','Visitor','Page Views',
                 'Entry','Bounce','Time Spent','SSD','PSSD','Memory Card','USB','PFS','PCD','PF','PD']]
scom['KPI']
#US와 global 빼고, 'RAW_KPI'를 위한 필요 정렬 완료

,Visit,Visit - PC,Visit - Mobile,Visitor,Page Views,Entry,Bounce,Time Spent,SSD,PSSD,Memory Card,USB,PFS,PCD,PF,PD
Item,,,,,,,,,,,,,,,,
ru,76453,40072,36381,67197,124597,67134,22397,170.475004,43006,4976,20260,10512,1200,10284,43245,40929
de,54359,33607,20752,49255,89188,37523,14674,103.686984,37357,8454,5338,3741,1624,10817,20960,33266
in,52327,16368,35959,47402,93106,35757,12605,147.348262,16745,17256,18394,0,3644,11307,24348,29653
cn,38841,31660,7181,34630,100278,17538,4211,154.139170,28138,4664,3930,2325,4099,25187,16162,21708
uk,29537,19520,10017,25726,55448,20692,7288,129.006622,15082,6426,6834,2014,259,6994,7700,23605
be,26457,11178,15279,20657,35187,25474,20091,171.010682,1840,25135,255,125,123,25552,1509,1450
tw,24527,7411,17116,20966,40814,22006,10017,140.099452,14389,5454,4305,609,656,8151,3670,18128
ca,23773,12015,11758,21917,38473,19591,7728,144.581841,8708,13482,1269,9555,1125,7241,2594,18840
es,23477,14684,8793,20727,36976,19680,5087,188.402488,17119,3691,3557,566,765,1,5194,20736


In [5]:
#똑같이 US의 데이터를 정제해보자
#모양이 쫌 다르니 달리 처리해준것들 유의
###############US 같은거 반복
#맨아래 time spent만 컬럼갯수달라 처리
sheet=['KPI','Referring Domain','Marketing Channel','URL','Product','Time Spent']

scom_us=Series()
for i in range(0,len(sheet)):
    scom_us[sheet[i]]= pd.read_excel('C:/Users/cnxk/Desktop/dashboard_prac/Scom_US_june.xlsx',

                              sheet_name = sheet[i], 

                              header = 1  )   
    
##dimension열 같은 필요없는 컬럼삭제
    col_not_dimension = []
    for j in scom_us[sheet[i]].columns:
        if bool(re.match('^Dimension',j)) == False:
            col_not_dimension.append(j)
    scom_us[sheet[i]]=scom_us[sheet[i]][col_not_dimension]
    
##자이제 Item컬럼의 빈행, 토탈행, low-traffic 행을 지워야한다.    
    scom_us[sheet[i]]=scom_us[sheet[i]].dropna(axis=1,how='all')#첫째열은 전체 빈행->삭제해주기
    scom_us[sheet[i]]=scom_us[sheet[i]].dropna(how='any')#NA값들어간 행 지우기
    
    
#필요없는 행들 지우기, 빈줄은 위에서 다지웠고
#KPI -> item=total
#'Referring Domain'->item.1=total
#'Marketing Channel'->item.1=total
#'URL'->item.1=total,(Low-traffic)
#'Product'->item.1=total
#'Time Spent'->item.1=Total

scom_us['KPI']=scom_us['KPI'].loc[ (scom_us['KPI']['Item']!='Total'),]
scom_us['Referring Domain']=scom_us['Referring Domain'].loc[ (scom_us['Referring Domain']['Item.1']!='Total'),]
scom_us['Marketing Channel']=scom_us['Marketing Channel'].loc[ (scom_us['Marketing Channel']['Item.1']!='Total'),]
scom_us['URL']=scom_us['URL'].loc[ (scom_us['URL']['Item.1']!='Total')
                                                      &(scom_us['URL']['Item.1']!=r'Low_Traffic'),]
scom_us['Product']=scom_us['Product'].loc[ (scom_us['Product']['Item.1']!='Total'),]
scom_us['Time Spent']=scom_us['Time Spent'].loc[ (scom_us['Time Spent']['Item.1']!='Total'),]
scom_us['Time Spent']=scom_us['Time Spent'].loc[ (scom_us['Time Spent']['[US] Memory - Visit']!=0),]
scom_us['Time Spent']
#프로덕트 원상태
US_product=scom_us['Product']['Item.1'].head(20).copy()

id_zip = zip(map['US_channel']['US Channel'], map['US_channel']['US - MST Mapping '])
id_zip =dict(id_zip )

scom_us['Marketing Channel']['Item.1']=scom_us['Marketing Channel']['Item.1'].map(id_zip)
scom_us['Marketing Channel']['Item.1']
scom_us['Marketing Channel']


a=scom_us['Marketing Channel'].pivot_table(values=['[US] Memory - Visit : Channel'],
                                                index=scom_us['Marketing Channel']['Item.1'],
                                                    aggfunc='sum')
scom_us['Marketing Channel']=a.reset_index().reset_index()
scom_us['Marketing Channel'].rename(columns = {'index' : 'Item'}, inplace = True)


C:\Users\cnxk\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  import sys


In [6]:

#US time spent 고치기
a=scom_us['Time Spent']['[US] Memory - Visit']
b=scom_us['Time Spent']['[US] Memory - Bounce']
c=scom_us['Time Spent']['Time Spent per Visit (seconds)']
sub=[x-y for x,y in zip (a,b)]
subT=[x*y for x,y in zip (sub,c)]

scom_us['Time Spent']['(V-B)']=sub
scom_us['Time Spent']['(V-B)*T']=subT
#이제 나라별 평균체류시간 구해야한다. 피벗
scom_us['Time Spent']

avg_duration= scom_us['Time Spent'].pivot_table(values=['(V-B)','(V-B)*T'],
                                                index=scom_us['Time Spent']['Item'],
                                                    aggfunc='sum')

avg_duration=avg_duration[avg_duration['(V-B)']!=0]

avg_duration['Spent Time']=[y/x for x,y in zip (avg_duration['(V-B)'],avg_duration['(V-B)*T'])]
avg_duration['Spent Time']



,Item,[US] Memory - Visit,[US] Memory - Visit : PC,[US] Memory - Visit : Mobile,[US] Memory - Visitor,[US] Memory - Page View,[US] Memory - Entry,[US] Memory - Bounce,[US] Memory - SSD,[US] Memory - PSSD,[US] Memory - Memory Card,[US] Memory - USB,[US] Memory - PFS,[MST] Memory - PCD,[US] Memory - PF,[US] Memory - PD,Time Spent
1,2020-06-01 00:00:00,216433,135538,80895,184852,594153,178171,40950,126420,41321,30730,13976,13219,0,73262,167057,160.914869


In [7]:
#원하는 컬럼순서로 맞춰주기
scom_us['KPI']['Time Spent']=avg_duration['Spent Time']
scom_us['KPI']=scom_us['KPI'].drop('Item',axis=1).set_index(Series('us'))

scom_us['KPI'].columns=['Visit','Visit - PC','Visit - Mobile','Visitor','Page Views',
                 'Entry','Bounce','SSD','PSSD','Memory Card','USB','PFS','PCD','PF','PD','Time Spent']
scom_us['KPI']['Time Spent']=np.array(avg_duration['Spent Time'])


scom_us['KPI']=scom_us['KPI'][['Visit','Visit - PC','Visit - Mobile','Visitor','Page Views',
                 'Entry','Bounce','Time Spent','SSD','PSSD','Memory Card','USB','PFS','PCD','PF','PD']]
scom_us['KPI']
#이로써 scom['Time Spent']와 통일해줌

#KPI모양은 다만들었다.
RAW_KPI=pd.concat([scom['KPI'],scom_us['KPI']])
RAW_KPI
#RAW_KPI 기본내용 완성 
#이후에 키값, 나라이름 매핑, 날짜 열 만들어야해

,Visit,Visit - PC,Visit - Mobile,Visitor,Page Views,Entry,Bounce,Time Spent,SSD,PSSD,Memory Card,USB,PFS,PCD,PF,PD
ru,76453,40072,36381,67197,124597,67134,22397,170.475004,43006,4976,20260,10512,1200,10284,43245,40929
de,54359,33607,20752,49255,89188,37523,14674,103.686984,37357,8454,5338,3741,1624,10817,20960,33266
in,52327,16368,35959,47402,93106,35757,12605,147.348262,16745,17256,18394,0,3644,11307,24348,29653
cn,38841,31660,7181,34630,100278,17538,4211,154.139170,28138,4664,3930,2325,4099,25187,16162,21708
uk,29537,19520,10017,25726,55448,20692,7288,129.006622,15082,6426,6834,2014,259,6994,7700,23605
be,26457,11178,15279,20657,35187,25474,20091,171.010682,1840,25135,255,125,123,25552,1509,1450
tw,24527,7411,17116,20966,40814,22006,10017,140.099452,14389,5454,4305,609,656,8151,3670,18128
ca,23773,12015,11758,21917,38473,19591,7728,144.581841,8708,13482,1269,9555,1125,7241,2594,18840
es,23477,14684,8793,20727,36976,19680,5087,188.402488,17119,3691,3557,566,765,1,5194,20736
th,20465,7652,12813,18378,31617,18968,5637,106.974041,8136,9370,2085,314,707,2105,3465,17312


In [8]:

#미국 채널 매핑해줘야하고, 상품 모양 바꿔줘야한다- 
#채널정리(None=>Other, Banner=>Display)->아래서 다른나라랑 묶어서 통째루 다해쥼 미국제품이름(GA=>G,BW=>,/*=> ,DA=>D, MZ들만 B삭제)


scom_us['Product']['Item.1']=scom_us['Product']['Item.1'].str.upper()
scom_us['Product']['Item.1'] = scom_us['Product']['Item.1'].str.replace("GA","G")

scom_us['Product']['Item.1'] = scom_us['Product']['Item.1'].str.replace("BW","").str.replace("/[a-zA-Z]*","").str.replace("DA","D")

product_clean=scom_us['Product']['Item.1'][scom_us['Product']['Item.1'].str.contains('^MZ')]
product_clean=product_clean.str.replace("B","")
product_clean_index=product_clean.index
product_clean_val=list(product_clean)


for i,j in zip(product_clean_index,product_clean_val):
    
    scom_us['Product'].loc[i,'Item.1']=j

###us 상품명 다 고쳤다
scom_us['Product']



,Item,Item.1,[US] Memory - Visit
3,2020-06-01 00:00:00,MZ-V7E1T0,12809
4,2020-06-01 00:00:00,MZ-V7S1T0,11610
5,2020-06-01 00:00:00,MZ-76E1T0,10686
6,2020-06-01 00:00:00,MU-PC1T0K,9880
7,2020-06-01 00:00:00,MZ-V7S500,8532
8,2020-06-01 00:00:00,MZ-76E500,7908
9,2020-06-01 00:00:00,MZ-V7E500,7371
10,2020-06-01 00:00:00,MB-ME512G,6200
11,2020-06-01 00:00:00,MZ-V7S2T0,6142
12,2020-06-01 00:00:00,MZ-76Q1T0,4910


In [9]:
scom_us['Referring Domain']=scom_us['Referring Domain'].set_index(Series(['us' for i in range(len(scom_us['Referring Domain']))]))
Series(['us' for i in range(len(scom_us['Referring Domain']))])

#kpi_detail에 들어갈 네가지 시트 rank, report, Item, value 만들어주기


scom['Referring Domain']=scom['Referring Domain'].reset_index(drop=True).reset_index().set_index('Item',drop=True)
scom['Referring Domain']['index']=scom['Referring Domain']['index']+np.array(1)
scom['Referring Domain'].rename(columns={'index':'Rank'},inplace=True)


scom['Marketing Channel']=scom['Marketing Channel'].reset_index(drop=True).reset_index().set_index('Item',drop=True)
scom['Marketing Channel']['index']=scom['Marketing Channel']['index']+np.array(1)
scom['Marketing Channel'].rename(columns={'index':'Rank'}, inplace=True)


scom['URL']=scom['URL'].reset_index(drop=True).reset_index().set_index('Item',drop=True)
scom['URL']['index']=scom['URL']['index']+np.array(1)
scom['URL'].rename(columns={'index':'Rank'}, inplace=True)


scom['Product']=scom['Product'].reset_index(drop=True).reset_index().set_index('Item',drop=True)
scom['Product']['index']=scom['Product']['index']+np.array(1)
scom['Product'].rename(columns={'index':'Rank'}, inplace=True)



####US 똑같이
#us로 인덱스 값 넣기

scom_us['Referring Domain']=scom_us['Referring Domain'].reset_index(drop=True).reset_index().set_index('Item',drop=True)
scom_us['Referring Domain']['index']=scom_us['Referring Domain']['index']+np.array(1)
scom_us['Referring Domain'].rename(columns={'index':'Rank'},inplace=True)
scom_us['Referring Domain'].rename(columns={'Item.1':'Item'},inplace=True)
scom_us['Referring Domain']=scom_us['Referring Domain'].set_index(Series(['us' for i in range(len(scom_us['Referring Domain']))]))

scom_us['Marketing Channel']=scom_us['Marketing Channel'].reset_index(drop=True).reset_index().set_index('Item',drop=True)
scom_us['Marketing Channel']['index']=scom_us['Marketing Channel']['index']+np.array(1)
scom_us['Marketing Channel'].rename(columns={'index':'Rank'}, inplace=True)
scom_us['Marketing Channel'].rename(columns={'Item.1':'Item'},inplace=True)
scom_us['Marketing Channel']=scom_us['Marketing Channel'].set_index(Series(['us' for i in range(len(scom_us['Marketing Channel']))]))

scom_us['URL']=scom_us['URL'].reset_index(drop=True).reset_index().set_index('Item',drop=True)
scom_us['URL']['index']=scom_us['URL']['index']+np.array(1)
scom_us['URL'].rename(columns={'index':'Rank'}, inplace=True)
scom_us['URL'].rename(columns={'Item.1':'Item'},inplace=True)
scom_us['URL']=scom_us['URL'].set_index(Series(['us' for i in range(len(scom_us['URL']))]))

scom_us['Product']=scom_us['Product'].reset_index(drop=True).reset_index().set_index('Item',drop=True)
scom_us['Product']['index']=scom_us['Product']['index']+np.array(1)
scom_us['Product'].rename(columns={'index':'Rank'}, inplace=True)
scom_us['Product'].rename(columns={'Item.1':'Item'},inplace=True)
scom_us['Product']=scom_us['Product'].set_index(Series(['us' for i in range(len(scom_us['Product']))]))


#컬럼이름 바꾸는 딕셔너리 생성

columns={"[MST] Memory_41 Sites + [MST] Memory - Visit : Channel":"Value",
         "[MST] Memory_41 Sites + [MST] Memory - Visit":"Value"}
#컬럼이름 Value로 바꾸기
#Report 컬럼 생성
#순위 매기기
scom['Referring Domain'].rename(columns=columns, inplace=True)
scom['Referring Domain']['Report']=np.array('Referring Domain')

scom['Marketing Channel'].rename(columns=columns, inplace=True)
scom['Marketing Channel']['Report']=np.array('Marketing Channel')

scom['URL'].rename(columns=columns, inplace=True)
scom['URL']['Report']=np.array('URL')

scom['Product'].rename(columns=columns, inplace=True)
scom['Product']['Report']=np.array('Product')

columns={"[US] Memory - Visit : Channel":"Value",
         "[US] Memory - Visit":"Value"}
    
scom_us['Referring Domain'].rename(columns=columns, inplace=True)
scom_us['Referring Domain']['Report']=np.array('Referring Domain')

scom_us['Marketing Channel'].rename(columns=columns, inplace=True)
scom_us['Marketing Channel']['Report']=np.array('Marketing Channel')

scom_us['URL'].rename(columns=columns, inplace=True)
scom_us['URL']['Report']=np.array('URL')

scom_us['Product'].rename(columns=columns, inplace=True)
scom_us['Product']['Report']=np.array('Product')

columns={"Item":"Item.1"}

scom_us['Referring Domain'].rename(columns=columns, inplace=True)

scom_us['Marketing Channel'].rename(columns=columns, inplace=True)

scom_us['URL'].rename(columns=columns, inplace=True)

scom_us['Product'].rename(columns=columns, inplace=True)


RAW_Detail=pd.concat([scom['Referring Domain'],scom_us['Referring Domain'],scom['Marketing Channel'],scom_us['Marketing Channel'],
                      scom['URL'],scom_us['URL'],scom['Product'],scom_us['Product']])
columns={"Item.1":"Item"}
RAW_Detail.rename(columns=columns, inplace=True)
RAW_Detail=RAW_Detail[['Report','Rank','Item','Value']]




#RAW_Country 만들기

RAW_Country=RAW_KPI['Visit'].sort_values(ascending=False)
RAW_Country=DataFrame(RAW_Country)
RAW_Country=RAW_Country.reset_index().reset_index()
RAW_Country['level_0']=RAW_Country['level_0']+np.array(1)
columns={"level_0":"Rank",
         "index":"Country"}
RAW_Country=RAW_Country.rename(columns=columns).set_index('Country')
RAW_Country

#나라매핑

    
CountryMap=map['Country_map']

CountryMap.set_index('Site',inplace=True)
CountryMap

RAW_KPI=pd.merge(CountryMap, RAW_KPI,left_index=True, right_index=True,how='inner')
#채널값들 바꿔주기
#us 제품명 바꿔주기
RAW_Detail=pd.merge(RAW_Detail, CountryMap,left_index=True, right_index=True, how='inner')

ChannelMap=map['Channel_map'][['Channel Detail_Raw','Channel Detail_Map']].set_index('Channel Detail_Raw')




RAW_Country=pd.merge(CountryMap, RAW_Country,left_index=True, right_index=True,how='inner').sort_values(by='Visit',ascending=False)

RAW_Country=RAW_Country[['Rank','Country','Visit']]

In [10]:
# 2020/7/9 
# 더해야할것?
# 글로벌 내용 넣기, RAW_KPI 합, RAW_Detail->채널별, 제품명
# RAW Detail 20개씩 자르기

#순위정하는거 이상함


RAW_KPI
GB=RAW_KPI.sum()
GB[0]='Global'
GB
RAW_KPI=RAW_KPI.append(GB,ignore_index=True)
# 글로벌 내용 넣기, RAW_KPI 합 생성완료
#이제 RAW_Detail  Item값 따라 합
#RAW_Detail 글로벌 내용구하기, 상위 20


In [11]:
#마케팅채널  none other로
#미국 채널 매핑해줘야

RAW_Detail['Item'] = RAW_Detail['Item'].str.replace("None","Other").str.replace("Banner","Display")
RAW_Detail[RAW_Detail['Country']=='US']




,Report,Rank,Item,Value,Country
us,Referring Domain,1,google.com,109974,US
us,Referring Domain,2,Typed/Bookmarked,69115,US
us,Referring Domain,3,bing.com,5872,US
us,Referring Domain,4,duckduckgo.com,2179,US
us,Referring Domain,5,yahoo.com,1216,US
...,...,...,...,...,...
us,Product,161,SM-T387VZKAVZW,1,US
us,Product,162,SM-T860NZBAXAR,1,US
us,Product,163,NP950SBE-X01US,1,US
us,Product,164,SM-T860NZNAXAR,1,US


In [12]:
RAW_Detail_GB= RAW_Detail.pivot_table(values=['Value'],
                                                    index=['Report', 'Item'],
                                                    aggfunc='sum')

RAW_Detail_GB=RAW_Detail_GB.sort_values(by = 'Value',ascending=False).groupby('Report').head(20)


RAW_Detail_GB=RAW_Detail_GB.sort_values(by=['Report', 'Value'],ascending=[True,False])

RAW_Detail_GB=RAW_Detail_GB.reset_index()

RAW_Detail_GB['Country']=np.array('Global')
RAW_Detail_GB
#RAW_Detail=RAW_Detail.drop(['Rank'],1)

RAW_Detail=RAW_Detail.sort_values(by = 'Value',ascending=False).groupby(['Report','Country']).head(20)
RAW_Detail=RAW_Detail.sort_values(by=['Country','Report','Value'],ascending=[True,True,False])

RAW_Detail=RAW_Detail.append(RAW_Detail_GB, ignore_index=True)


In [13]:

##KPI spent time 글로벌 수정

a=RAW_KPI['Visit'].tail(1)-RAW_KPI['Bounce'].tail(1)
b=sum([(x-y)*z for x,y,z in zip (RAW_KPI['Visit'],RAW_KPI['Bounce'],RAW_KPI['Time Spent'])])#해당국가에서 전체 

RAW_KPI['Time Spent'][RAW_KPI['Country']=='Global']=b/a/42

#US_product 원래이름으로 돌리기
a=RAW_Detail[(RAW_Detail['Country']=='US')&(RAW_Detail['Report']=='Product')].index
US_product=list(US_product)

for i in a:
    RAW_Detail.iloc[i,1]=US_product[i-a[0]]

C:\Users\cnxk\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [14]:
writer = pd.ExcelWriter('C:/Users/cnxk/Desktop/dashboard_prac/clean_data_june.xlsx', engine='xlsxwriter')
RAW_Country.to_excel(writer , sheet_name = 'RAW_Country')
RAW_Detail.to_excel(writer , sheet_name = 'RAW_Detail')
RAW_KPI.to_excel(writer , sheet_name = 'RAW_KPI')
writer.save()

In [ ]:
RAW_Detail[RAW_Detail['Country']=='Global']

In [ ]:
RAW_Detail[RAW_Detail['Country']=='US']